In [129]:
%load_ext autoreload
%autoreload 2
from EPECinterface import *
from gurobipy import *
import itertools
import csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
def printVars(vv, prefix = "", tol=1e-6):
    for vvv in vv:
        if abs(vv[vvv].X) >= tol:
            print(prefix,vvv,"\t---\t",vv[vvv].X)

# Sets

In [131]:
dirty = ["coal", "gas"]
green = ["wind", "solar"]

countries = ["c1", "c2"]
producers = ["p1_1", "p1_2", "p2_1", "p2_2"]

domesticity = tuplelist([("c1","p1_1"), ("c1","p1_2"),
               ("c2","p2_1"), ("c2","p2_2")
              ])

scenario = ["s"+str(i+1) for i in range(2
                                       )]

data = inputData(dirty, green, countries, producers, domesticity, scenario)

In [132]:
mylist = readInputFromFile("inputs.csv")
baseData = mylist[0]
data = inputData(dirty, green, countries, producers, domesticity, scenario)
data.buildFromList(baseData)

In [133]:
data.cciInt = 0
data.cciSlope = 2

In [134]:
# data.dirty = ['gas']
# data.green = ['wind']
# data.energy = ['gas','wind']
# data.scenario = ['s1']
# data.probability = {'s1':1}
for pp in data.producers:
    data.InitCredits[pp] = 0

# Complementarity Problem Formulation

## Variables

In [135]:
M = Model()
# ## Variables
# ### Follower Variables 
PRODUCTION = M.addVars(data.producers, data.energy, data.scenario, 
                       vtype=GRB.CONTINUOUS, name = "PROD")
INVESTMENT = M.addVars(data.producers, data.energy, 
                       vtype=GRB.CONTINUOUS, name = "INV") 
FOLL_CARB_BUY = M.addVars(data.producers, lb = -GRB.INFINITY, 
                              name = "FOLL_CARB_BUY")

### All data.dirty investments must be 0 
for ee in data.dirty:
    for x in INVESTMENT.select("*",ee):
        x.ub = 0


# ### Leader Variables 

TOTAL_INV = M.addVars(data.countries, data.energy, name = "TOTAL_INV")
TOTAL_EMIT = M.addVars(data.countries, data.scenario, name = "TOTAL_EMIT")
C_PROD = M.addVars(data.countries, data.scenario, name="C_PROD")

# ### Duals 
D_INFRALIMIT = M.addVars(data.producers, data.energy, data.scenario,
                        name = "D_INFRA")
D_EMITLIMIT = M.addVars(data.producers, data.scenario, name = "D_EMIT")

# ### Binary variables 
B_PRODUCTION= M.addVars(data.producers, data.energy, data.scenario, 
                       vtype= GRB.BINARY, name = "B_Prod")
B_INVESTMENT= M.addVars(data.producers, data.energy, 
                       vtype= GRB.BINARY, name = "B_Invest")
B_CARBON = M.addVars(data.producers, vtype = GRB.BINARY, name = "B_Carbon")
B_INFRALIM = M.addVars(data.producers, data.energy, data.scenario, 
                      vtype = GRB. BINARY, name = "B_InfraLim")
B_EMITLIM = M.addVars(data.producers, data.scenario,  
                      vtype = GRB. BINARY, name = "B_EmitLim")

## Constraints

In [136]:
# Constraints
# ### Follower KKT conditions 

# eq_infraLimit
for pp,ee,ss in itertools.product(data.producers,data.energy,data.scenario):
    eqn = 0
    if ee in data.green:
        eqn = data.CapacityFactor[pp, ee, ss]*(INVESTMENT[pp,ee]+data.InitCapacity[pp, ee]) - PRODUCTION[pp, ee, ss]
    else:
        eqn = INVESTMENT[pp,ee]+data.InitCapacity[pp, ee]- PRODUCTION[pp, ee, ss]
    M.addConstr(eqn >= 0,name = "eq_infraLimit"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_INFRALIM[pp,ee,ss],0,eqn ==0, name = "eq_infraLimit1"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_INFRALIM[pp,ee,ss],1,D_INFRALIMIT[pp,ee,ss] ==0, name = "eq_infraLimit2"+"_".join([pp,ee,ss]))

# eq_emitLimit
for pp,ss in itertools.product(data.producers, data.scenario):
    eqn = data.InitCredits[pp] + FOLL_CARB_BUY[pp] - quicksum(data.Emission[ee]*PRODUCTION[pp,ee,ss] for ee in data.dirty)
    M.addConstr(eqn >= 0,name = "eq_emitLimit"+"_".join([pp,ss]))
    M.addGenConstrIndicator(B_EMITLIM[pp,ss],0,eqn ==0, name = "eq_emitLimit1"+"_".join([pp,ss]))
    M.addGenConstrIndicator(B_EMITLIM[pp,ss],1,D_EMITLIMIT[pp,ss] ==0, name = "eq_emitLimit2"+"_".join([pp,ss]))

# eq_countryProduction
for ss in data.scenario:
    for cc in data.countries:
        eqn = 0
        for pp in data.producers:
            if (cc,pp) in data.domesticity:
                eqn = eqn + quicksum(PRODUCTION[pp,ee,ss] for ee  in data.energy)
        M.addConstr(eqn == C_PROD[cc,ss], name="eq_countryProduction"+"_".join([cc,ss]))

# eq_production
for pp,ee,ss in itertools.product(data.producers,data.energy,data.scenario):
    eqn = data.probability[ss]*(
        data.LinProdCost[pp,ee] + data.QuadProdCost[pp,ee]*PRODUCTION[pp,ee,ss] -
        quicksum(data.DemInt[cc[0],ss] - data.DemSlope[cc[0],ss]*C_PROD[cc[0],ss] for cc in data.domesticity.select("*",pp)) +
        quicksum(data.DemSlope[cc[0],ss]*PRODUCTION[pp,ee2,ss] for cc in data.domesticity.select("*",pp) for ee2 in data.energy) 
    )+  D_INFRALIMIT[pp,ee,ss] + data.Emission[ee]*D_EMITLIMIT[pp,ss]

    M.addConstr(eqn >= 0,name = "eq_PRODUCTION"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_PRODUCTION[pp,ee,ss],0,eqn ==0, name = "eq_PRODUCTION1"+"_".join([pp,ee,ss]))
    M.addGenConstrIndicator(B_PRODUCTION[pp,ee,ss],1,PRODUCTION[pp,ee,ss] ==0, name = "eq_PRODUCTION2"+"_".join([pp,ee,ss]))

# eq_investment
for pp,ee in itertools.product(data.producers,data.green):
    eqn = data.LinInvCost[pp,ee] + data.QuadInvCost[pp,ee]*INVESTMENT[pp,ee] - quicksum(data.CapacityFactor[pp,ee,ss]*D_INFRALIMIT[pp,ee,ss] for ss in data.scenario)
    M.addConstr(eqn >= 0,name = "eq_INVESTMENT"+"_".join([pp,ee]))
    M.addGenConstrIndicator(B_INVESTMENT[pp,ee],0,eqn ==0, name = "eq_INVESTMENT1"+"_".join([pp,ee]))
    M.addGenConstrIndicator(B_INVESTMENT[pp,ee],1,INVESTMENT[pp,ee] ==0, name = "eq_INVESTMENT2"+"_".join([pp,ee]))

# eq_carbonPurchase
for pp in data.producers:
    eqn = (data.cciInt + data.cciSlope*quicksum(FOLL_CARB_BUY[ppp] for ppp in data.producers)) + data.cciSlope*FOLL_CARB_BUY[pp] -quicksum(D_EMITLIMIT[pp,ss] for ss in data.scenario)
    M.addConstr(eqn >= 0, name = "eq_carbonPurchase"+str(pp))
    M.addGenConstrIndicator(B_CARBON[pp],0,eqn ==0, name = "eq_CARBON1"+"_"+str(pp))
    M.addGenConstrIndicator(B_CARBON[pp],1,FOLL_CARB_BUY[pp] ==0, name = "eq_CARBON2"+"_"+str(pp))

# Total data.Emission
for ss in data.scenario:
    eqn = {cc:0 for cc in data.countries}
    for (cc,pp) in data.domesticity:
        eqn[cc] = eqn[cc] + quicksum(data.Emission[ee]*PRODUCTION[pp,ee,ss] for ee in data.energy)
    M.addConstrs((TOTAL_EMIT[cc,ss] - eqn[cc]==0 for cc in data.countries), name="TotalEmission"+str(ss))

# Total data.Investment
for ee in data.green:
    eqn = {cc:0 for cc in data.countries}
    for (cc,pp) in data.domesticity:
        eqn[cc] = eqn[cc] + INVESTMENT[pp,ee] 
    M.addConstrs((TOTAL_INV[cc,ee] - eqn[cc]==0 for cc in data.countries), name="TotalInv"+"_".join([cc,ee]))

M.Params.NonConvex=2
M.Params.Presolve = 2
M.Params.CutPasses = 1
M.Params.PreQLinearize = 1
M.Params.Threads = 2
M.update()


Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter CutPasses to 1
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter PreQLinearize to 1
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0


In [137]:
M.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 96 rows, 200 columns and 412 nonzeros
Model fingerprint: 0x5f8473d4
Model has 168 general constraints
Variable types: 108 continuous, 92 integer (92 binary)
Coefficient statistics:
  Matrix range     [1e-01, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 5e+02]
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Presolve added 72 rows and 34 columns
Presolve time: 0.00s
Presolved: 168 rows, 234 columns, 598 nonzeros
Presolved model has 66 SOS constraint(s)
Variable types: 146 continuous, 88 integer (88 binary)
Presolve removed 108 rows and 146 columns
Presolved: 60 rows, 88 columns, 232 nonzeros


Root relaxation: objective 0.000000e+00, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  

In [123]:
prodObj = {}
prodCost = {} # (pp,ss)
prod = {} #(pp, ee)
invCost = {}
carbCost = {}
energPrice = {}
revenue = {}

for pp in data.producers:
    for ss in data.scenario:
        prodCost[pp,ss] = 0
        prod[pp,ss] = 0
        for ee in data.energy:
            prodCost[pp,ss] += data.LinProdCost[pp,ee]*PRODUCTION[pp,ee,ss].X*data.probability[ss]
            prodCost[pp,ss] += data.QuadProdCost[pp,ee]*PRODUCTION[pp,ee,ss].X*PRODUCTION[pp,ee,ss].X*data.probability[ss]
            prod[pp,ss] += PRODUCTION[pp,ee,ss].X*data.probability[ss]
            
    invCost[pp] = 0
    for gg in data.green:
        invCost[pp] += data.LinInvCost[pp,gg]*INVESTMENT[pp,ee].X
        invCost[pp] += data.QuadInvCost[pp,gg]*INVESTMENT[pp,ee].X*INVESTMENT[pp,ee].X
    
    carbCost[pp] = (data.cciInt + data.cciSlope*sum([FOLL_CARB_BUY[ppp].X for ppp in data.producers]))*FOLL_CARB_BUY[pp].X

for cc,ss in itertools.product(data.countries,data.scenario):
    energPrice[cc,ss] = data.DemInt[cc,ss] - data.DemSlope[cc,ss] * C_PROD[cc,ss].X

for pp in data.producers:
    cntry = data.domesticity.select("*",pp)[0][0]
    revenue[pp] = 0
    for ss in data.scenario:
        revenue[pp] += energPrice[cc,ss]*prod[pp,ss]*data.probability[ss]
    
        
    prodObj[pp] = revenue[pp] + carbCost[pp] + invCost[pp] + sum(prodCost[pp,ss] for ss in data.scenario)

In [124]:
carbCost

{'p1_1': 23506.135936474035,
 'p1_2': 23506.135936474035,
 'p2_1': 23501.988709827598,
 'p2_2': 23501.988709827583}

In [126]:
FOLL_CARB_BUY

{'p1_1': <gurobi.Var FOLL_CARB_BUY[p1_1] (value 54.20817481332834)>,
 'p1_2': <gurobi.Var FOLL_CARB_BUY[p1_2] (value 54.208174813328334)>,
 'p2_1': <gurobi.Var FOLL_CARB_BUY[p2_1] (value 54.19861077491521)>,
 'p2_2': <gurobi.Var FOLL_CARB_BUY[p2_2] (value 54.19861077491518)>}

In [128]:
(data.cciInt + data.cciSlope*sum([FOLL_CARB_BUY[ppp].X for ppp in data.producers]))

433.62714235297415